In [82]:
import duckdb

Setup File Path & Object Name di MinIO

In [83]:
localPath = {
    'bekasi': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Bekasi.csv',
    'depok': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Depok.csv'
    }

minioObjectName = {
    'bekasi': 'JawaBarat/dataBekasi.csv',
    'depok': 'JawaBarat/dataDepok.csv'
    }

MinIO Setup

In [84]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

duckdb Setup

In [85]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")

Minio Connection

In [86]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';") # WAJIB untuk MinIO/Local

Full EndPoint URL

In [87]:
fullEndPointURLBekasi = f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}"
fullEndPointURLDepok = f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}"

In [88]:
loadBekasi = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['bekasi']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURLBekasi}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadDepok = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['depok']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURLDepok}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""

In [89]:
# --- 2. Jalankan Proses Migrasi ---
try:
    con.sql(loadBekasi)
    con.sql(loadDepok)
    print(f"\n✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO")
except Exception as e:
    print(f"\n❌ Terjadi error saat migrasi: {e}")


✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO


In [90]:
con.close()